```{contents}
```
## Partial State Updates 

In LangGraph, **Partial State Updates** allow a node to return **only the subset of the global state it intends to modify**, rather than the entire state object.
LangGraph then **merges these updates** into the existing global state using **state reducers**.

This enables **modular design, safety, performance, and scalability** in complex LLM workflows.

---

### **1. Motivation and Intuition**

Without partial updates, every node would need to construct the **entire state** on every step.
This quickly becomes fragile and inefficient.

| Problem          | Consequence                      |
| ---------------- | -------------------------------- |
| State copying    | Large memory overhead            |
| Tight coupling   | Nodes depend on unrelated fields |
| High error risk  | Accidental state deletion        |
| Poor scalability | State grows uncontrollably       |

Partial updates solve this by allowing:

> **"Update only what you change."**

---

### **2. Conceptual Model**

```
Current Global State:
{ A, B, C, D }

Node returns:
{ B, D }

Reducer merges:
{ A, B', C, D' }
```

Only modified fields are replaced.
Unmentioned fields remain intact.

---

### **3. How Partial State Updates Work**

A node returns a **dictionary containing only modified keys**.

```python
def node_fn(state):
    new_plan = compute_plan(state["goal"])
    return {"plan": new_plan}
```

LangGraph merges this into the existing global state:

```python
state["plan"] = new_plan
# all other keys unchanged
```

---

### **4. Why This Matters**

| Property        | Benefit                         |
| --------------- | ------------------------------- |
| Modularity      | Nodes become independent        |
| Safety          | Cannot accidentally erase state |
| Performance     | Smaller updates & checkpoints   |
| Maintainability | Easy to reason about changes    |
| Scalability     | Large systems remain stable     |

---

### **5. Partial Updates vs Full Updates**

| Feature     | Partial Update | Full State Return |
| ----------- | -------------- | ----------------- |
| Size        | Small          | Large             |
| Risk        | Low            | High              |
| Coupling    | Loose          | Tight             |
| Performance | High           | Low               |
| Correctness | Strong         | Fragile           |

---

### **6. Example: Clean Multi-Step Workflow**



In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

class State(TypedDict):
    goal: str
    plan: str
    result: str

def planner(state):
    plan = f"Plan for: {state['goal']}"
    return {"plan": plan}

def executor(state):
    result = f"Executing: {state['plan']}"
    return {"result": result}

builder = StateGraph(State)
builder.add_node("planner", planner)
builder.add_node("executor", executor)
builder.set_entry_point("planner")
builder.add_edge("planner", "executor")
builder.add_edge("executor", END)

graph = builder.compile()
print(graph.invoke({"goal": "Build agent"}))


{'goal': 'Build agent', 'plan': 'Plan for: Build agent', 'result': 'Executing: Plan for: Build agent'}




**Final State**

```python
{
  "goal": "Build agent",
  "plan": "Plan for: Build agent",
  "result": "Executing: Plan for: Build agent"
}
```

Each node touched only its own field.

---

### **7. Partial Updates with Reducers**

Reducers define **how updates are merged**.

```python
from typing_extensions import Annotated
from langgraph.graph import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]
```

Now a node can return:

```python
return {"messages": [new_message]}
```

Reducer appends instead of replacing.

---

### **8. Partial Updates in Cycles & Agents**

In loops and multi-agent systems:

* Each agent updates only its own fields
* Shared memory grows in a controlled manner
* Checkpoints remain compact
* Long-running workflows remain stable

---

### **9. Anti-Patterns to Avoid**

| Anti-Pattern              | Why Bad           |
| ------------------------- | ----------------- |
| Returning entire state    | Risk of data loss |
| Overwriting shared memory | Destroys history  |
| Unnecessary updates       | Wastes memory     |
| Tight state coupling      | Hard to debug     |

---

### **10. Mental Model**

> **Partial State Update = transactional write**
> **Full State Return = memory overwrite**

Only touch what you intend to change.

---

### **11. When to Use Partial State Updates**

Use partial updates for:

* Plans
* Decisions
* Tool results
* Agent messages
* Progress indicators
* Long-term memory entries

Avoid full state returns unless performing controlled state migration.

---

### **12. Key Design Principle**

> **Each node owns only the fields it updates.**

This principle yields **robust, safe, production-grade LangGraph systems**.